In [ ]:
from matplotlib.pylab import LinAlgError
import numpy as np
import pandas as pd
from scipy.special import logit, expit 
import statsmodels.api as sm
from statsmodels.regression.mixed_linear_model import MixedLM
import tqdm

import warnings
from functools import wraps
from contextlib import contextmanager
from statsmodels.tools.sm_exceptions import ConvergenceWarning

class WarningCounter:
    def __init__(self):
        self.count = 0
    
    def __call__(self, message, category, filename, lineno, file=None, line=None):
        self.count += 1

@contextmanager
def count_warnings(warning_type=Warning):
    # Create counter and set as warning handler
    counter = WarningCounter()
    old_showwarning = warnings.showwarning
    warnings.showwarning = counter
    
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings('always', category=warning_type)
            yield counter
    finally:
        # Restore original warning handler
        warnings.showwarning = old_showwarning

def simulate_study(n_participants, n_total_items, n_items_per_participant, 
                    participant_var=0.5, item_var=0.3, baseline_preference=0.6):
    """
    Simulate binary choices in a balanced incomplete design.
    """
    # Convert probability to log odds
    log_odds = logit(baseline_preference)
    
    # Generate random effects
    participant_effects = np.random.normal(0, np.sqrt(participant_var), n_participants)
    item_effects = np.random.normal(0, np.sqrt(item_var), n_total_items)
    
    data = []
    
    # For each participant
    for p in range(n_participants):
        # Randomly select items
        selected_items = np.random.choice(n_total_items, n_items_per_participant, replace=False)
        
        # Calculate probabilities
        etas = log_odds + participant_effects[p] + item_effects[selected_items]
        probs = expit(etas)
        
        # Generate choices
        choices = np.random.binomial(1, probs)
        
        # Store results
        for item, choice in zip(selected_items, choices):
            data.append({'participant': p, 'item': item, 'choice': choice})
    
    return pd.DataFrame(data)

In [15]:
simulated_5 = simulate_study(5, 50, 50)
simulated_5.head(5)

,participant,item,choice
0,0,14,0
1,0,18,0
2,0,47,1
3,0,43,1
4,0,26,0


In [16]:
simulated_5.groupby(['participant', 'choice']).size().unstack()

choice,0,1
participant,,
0,18,32
1,25,25
2,28,22
3,20,30
4,20,30


In [30]:
def power_analysis_mixed(n_participants, n_sims=1, n_total_items=50, n_items_per_participant=50, participant_var=.5, item_var=.3):
    """Power analysis using mixed effects model"""
    significant_count = 0
    fixed_effects = []
    
    with count_warnings(ConvergenceWarning) as counter:
        for _ in tqdm.tqdm(range(n_sims)):
            # Generate data
            data = simulate_study(n_participants=n_participants,
                                    n_total_items=n_total_items,
                                    n_items_per_participant=n_items_per_participant,
                                    participant_var=participant_var,
                                    item_var=item_var)

            # Fit mixed model
            # Using participant as random effect (no case-effects)
            # constant term to test if it's different from 0.5
            
            model = MixedLM(data.choice, 
                            exog=sm.add_constant(np.ones(len(data))),
                            groups=data.participant)
            
            try:
                result = model.fit()
            except LinAlgError:
                counter.count += 1
                continue
            
            # Store fixed effect (intercept)
            fixed_effects.append(result.params.iloc[0])
            
            # Test if significantly different from 0.5
            # Using robust standard errors
            if result.pvalues.iloc[0] < 0.05:
                significant_count += 1
    
    return {
        'power': significant_count / n_sims,
        'avg_effect': np.mean(fixed_effects),
        'effect_sd': np.std(fixed_effects),
        'failed_convergence': counter.count
    }

In [33]:
participant_counts = [10, 15, 20, 25, 30]
variance_scenarios = [
    {'name': 'Low variance', 'p_var': 0.3, 'i_var': 0.2},
    {'name': 'Medium variance', 'p_var': 0.5, 'i_var': 0.4},
    {'name': 'High variance', 'p_var': 0.8, 'i_var': 0.6}
]

rows = []
for scenario in variance_scenarios:
    for n in participant_counts:
        for n_items_per_participant in [60]:
            power_res = power_analysis_mixed(n_participants=n, n_sims=100, n_total_items=n_items_per_participant, n_items_per_participant=n_items_per_participant, participant_var=scenario['p_var'], item_var=scenario['i_var'])
            rows.append([scenario['name'], n, n_items_per_participant, n_items_per_participant, power_res['power'], power_res['avg_effect'], power_res['effect_sd'], power_res['failed_convergence']])

100%|██████████| 100/100 [00:12<00:00,  7.95it/s]


In [34]:
import pandas as pd
pd.DataFrame(rows, columns=['Scenario', 'Participants', 'Total items', 'Items per participant', 'Power', 'Avg effect', 'Effect SD', 'Failed convergence'])

,Scenario,Participants,Total items,Items per participant,Power,Avg effect,Effect SD,Failed convergence
0,Low variance,10,60,60,1.0,0.587700,0.040942,135
1,Low variance,15,60,60,1.0,0.594533,0.039698,146
2,Low variance,20,60,60,1.0,0.594258,0.036562,206
3,Low variance,25,60,60,1.0,0.592520,0.032988,181
4,Low variance,30,60,60,1.0,0.594517,0.027183,201
5,Medium variance,10,60,60,1.0,0.582817,0.048868,70
6,Medium variance,15,60,60,1.0,0.586722,0.043150,76
7,Medium variance,20,60,60,1.0,0.581008,0.037217,94
8,Medium variance,25,60,60,1.0,0.586120,0.034117,137
9,Medium variance,30,60,60,1.0,0.586039,0.029744,92


In [35]:
participant_counts = [10, 15, 20]
variance_scenarios = [
    {'name': 'Low variance', 'p_var': 0.3, 'i_var': 0.2},
    {'name': 'Medium variance', 'p_var': 0.5, 'i_var': 0.4},
    {'name': 'High variance', 'p_var': 0.8, 'i_var': 0.6}
]

rows = []
for scenario in variance_scenarios:
    for n in participant_counts:
        for n_items_per_participant in [50]:
            power_res = power_analysis_mixed(n_participants=n, n_sims=100, n_total_items=n_items_per_participant, n_items_per_participant=n_items_per_participant, participant_var=scenario['p_var'], item_var=scenario['i_var'])
            rows.append([scenario['name'], n, n_items_per_participant, n_items_per_participant, power_res['power'], power_res['avg_effect'], power_res['effect_sd'], power_res['failed_convergence']])

100%|██████████| 100/100 [00:29<00:00,  3.45it/s]


In [36]:
pd.DataFrame(rows, columns=['Scenario', 'Participants', 'Total items', 'Items per participant', 'Power', 'Avg effect', 'Effect SD', 'Failed convergence'])

,Scenario,Participants,Total items,Items per participant,Power,Avg effect,Effect SD,Failed convergence
0,Low variance,10,50,50,0.96,0.588125,0.047522,182
1,Low variance,15,50,50,0.89,0.593573,0.039997,175
2,Low variance,20,50,50,0.89,0.583337,0.032977,180
3,Medium variance,10,50,50,0.99,0.592081,0.054141,169
4,Medium variance,15,50,50,0.97,0.585663,0.044469,275
5,Medium variance,20,50,50,0.98,0.581357,0.040312,171
6,High variance,10,50,50,0.99,0.575576,0.063395,145
7,High variance,15,50,50,1.00,0.586187,0.050407,160
8,High variance,20,50,50,0.99,0.583818,0.041996,230


In [37]:
participant_counts = [10, 15, 20, 25, 30, 35]
variance_scenarios = [
    {'name': 'Low variance', 'p_var': 0.3, 'i_var': 0.2},
]

rows = []
for scenario in variance_scenarios:
    for n in participant_counts:
        for n_items_per_participant in [50]:
            power_res = power_analysis_mixed(n_participants=n, n_sims=100, n_total_items=n_items_per_participant, n_items_per_participant=n_items_per_participant, participant_var=scenario['p_var'], item_var=scenario['i_var'])
            rows.append([scenario['name'], n, n_items_per_participant, n_items_per_participant, power_res['power'], power_res['avg_effect'], power_res['effect_sd'], power_res['failed_convergence']])

100%|██████████| 100/100 [00:37<00:00,  2.69it/s]


In [38]:
pd.DataFrame(rows, columns=['Scenario', 'Participants', 'Total items', 'Items per participant', 'Power', 'Avg effect', 'Effect SD', 'Failed convergence'])

,Scenario,Participants,Total items,Items per participant,Power,Avg effect,Effect SD,Failed convergence
0,Low variance,10,50,50,0.95,0.592358,0.042250,169
1,Low variance,15,50,50,0.90,0.593467,0.034658,207
2,Low variance,20,50,50,0.89,0.589989,0.030451,186
3,Low variance,25,50,50,0.85,0.585666,0.025034,113
4,Low variance,30,50,50,0.89,0.590390,0.026913,130
5,Low variance,35,50,50,0.78,0.589355,0.024191,143
